# What Changed Since 2024? - Part 1B

This notebook explains the key changes and improvements we've made to modernize the original RAG application for 2025. Understanding these changes will help you appreciate why certain technologies have evolved and what the current best practices are.

## 🚨 Major Change #1: LangServe is Deprecated

### What was LangServe?
LangServe was a tool that automatically created REST APIs from LangChain chains. It seemed convenient because you could deploy a chain with just one line:

```python
# Old approach (2024)
from langserve import add_routes
add_routes(app, final_chain, path="/rag")
```

### Why is it deprecated?
As of 2024, LangChain officially recommends **LangGraph Platform** instead of LangServe for new projects. The reasons:

1. **Limited Flexibility**: LangServe was too rigid for real-world applications
2. **Performance Issues**: Added unnecessary layers of abstraction
3. **Maintenance**: The LangChain team shifted focus to LangGraph
4. **Industry Trend**: Direct FastAPI implementations became the standard

### Modern Alternative: Direct FastAPI
Instead of LangServe, we now use direct FastAPI endpoints, which gives us:
- ✅ Full control over API behavior
- ✅ Better error handling
- ✅ Custom validation
- ✅ Streaming responses
- ✅ Better documentation

## 💰 Major Change #2: OpenAI Embedding Model Update

### Old Model (2024): `text-embedding-ada-002`
```python
# Old approach
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
# Cost: $0.0001 per 1k tokens
```

### New Model (2025): `text-embedding-3-small`
```python
# Modern approach
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
# Cost: $0.00002 per 1k tokens (5x cheaper!)
```

### Why the change?
- **💵 Cost Reduction**: 5x cheaper than ada-002
- **📈 Better Performance**: Improved accuracy on most benchmarks
- **🔮 Future Proof**: Part of OpenAI's latest embedding family

**Note**: `text-embedding-ada-002` is NOT deprecated, but the new models are much better value.

## 🐍 Major Change #3: Python Version Update

### Old Version: Python 3.11.4
```toml
# Old pyproject.toml
python = ">=3.11,<3.12"
```

### New Version: Python 3.13.3
```toml
# Modern pyproject.toml
python = ">=3.13,<3.14"
```

### Benefits of Python 3.13:
- **⚡ Performance**: Up to 15% faster than 3.11
- **🔒 Security**: Latest security patches
- **🛠️ New Features**: Better error messages, improved typing
- **📦 Compatibility**: All major AI libraries now support 3.13

## 🎯 Major Change #4: Simplified Chain Structure

### Old Approach (with LangServe quirks):
```python
# Old rag_chain.py - needed special input types for LangServe
from typing import TypedDict
from operator import itemgetter

class RagInput(TypedDict):
    question: str

final_chain = (
    {
    "context": (itemgetter("question") | vector_store.as_retriever()),
    "question": itemgetter("question")
    }
    | ANSWER_PROMPT
    | llm
    | StrOutputParser()
).with_types(input_type=RagInput)  # LangServe requirement
```

### Modern Approach (Direct and Clean):
```python
# Modern rag_chain.py - much cleaner!
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)
```

### Why this is better:
- ✅ **Simpler**: No need for TypedDict or itemgetter
- ✅ **More Readable**: Clear data flow
- ✅ **Flexible**: Works with any input format
- ✅ **Standard**: Follows current LangChain patterns

## 🤖 Major Change #5: Modern LLM Model

### Old Model: `gpt-4-1106-preview`
```python
# Old approach
llm = ChatOpenAI(temperature=0, model='gpt-4-1106-preview', streaming=True)
```

### New Model: `gpt-4o-mini`
```python
# Modern approach
llm = ChatOpenAI(temperature=0, model='gpt-4o-mini', streaming=True)
```

### Why `gpt-4o-mini`?
- **💰 Much Cheaper**: 60x cheaper than GPT-4
- **⚡ Faster**: Lower latency responses
- **🎯 RAG Optimized**: Specifically good for Q&A tasks
- **📊 Great Performance**: 82% as capable as GPT-4o on most tasks

## 📦 Major Change #6: Cleaner Dependencies

### Old Dependencies (2024):
```toml
# Had to constrain Python version due to unstructured package
python = ">=3.11,<3.12"  # Forced constraint
pydantic = "<2"          # LangServe requirement
langserve = {extras = ["server"], version = ">=0.0.30"}
uvicorn = "^0.23.2"      # Old version
```

### Modern Dependencies (2025):
```toml
# Clean, modern versions
python = ">=3.13,<3.14"         # Latest stable
fastapi = "^0.115.0"            # Direct FastAPI
uvicorn = "^0.32.0"             # Latest uvicorn
langchain-openai = "^0.2.0"     # Modular imports
# No pydantic constraints!
# No langserve dependency!
```

### Benefits:
- ✅ **Fewer Conflicts**: No version constraint wars
- ✅ **Faster Installs**: Fewer dependencies to resolve
- ✅ **Better Security**: Latest versions with security patches
- ✅ **Future Proof**: Using actively maintained packages

## 🔧 Major Change #7: Better Error Handling

### Old Server (LangServe - Limited Control):
```python
# Old server.py - minimal control
from langserve import add_routes
from app.rag_chain import final_chain

app = FastAPI()
add_routes(app, final_chain, path="/rag")  # Black box!
```

### Modern Server (Full Control):
```python
# Modern server.py - complete control
@app.post("/query", response_model=QueryResponse)
async def query_documents(request: QueryRequest):
    try:
        answer = await rag_chain.ainvoke(request.question)
        return QueryResponse(answer=answer)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error: {str(e)}")

# Plus streaming support!
@app.post("/stream")
async def stream_query(request: QueryRequest):
    # Custom streaming implementation
```

### What we gained:
- ✅ **Custom Error Messages**: Helpful debugging info
- ✅ **Streaming Support**: Real-time responses
- ✅ **Request Validation**: Proper Pydantic models
- ✅ **Health Checks**: Monitoring endpoints
- ✅ **API Documentation**: Automatic OpenAPI docs

## 📁 Major Change #8: Document Processing Simplification

### Old Code (Required Flattening):
```python
# Old rag_load_and_process.py
docs = loader.load()

# This weird flattening step was needed:
flattened_docs = [doc[0] for doc in docs if doc]
chunks = text_splitter.split_documents(flattened_docs)
```

### Modern Code (Direct Processing):
```python
# Modern rag_load_and_process.py
docs = loader.load()

# No flattening needed anymore!
chunks = text_splitter.split_documents(docs)
```

### Why the change?
- **🐛 Bug Fix**: The document loader structure was fixed in newer versions
- **🧹 Cleaner**: No more mysterious flattening step
- **📖 Readable**: Code is self-explanatory

## 🎓 Educational Benefits of These Changes

### For Beginner Students:
1. **Learn Modern Patterns**: You're learning current industry standards, not outdated approaches
2. **Understand Evolution**: See how AI development practices evolve rapidly
3. **Cost Awareness**: Learn about the economic factors in AI development
4. **Future Skills**: FastAPI is more widely used than LangServe ever was

### Key Takeaways:
- 🚀 **AI moves fast**: What's cutting-edge today might be deprecated in 12 months
- 💡 **Simplicity wins**: Direct approaches often beat complex abstractions
- 💰 **Cost matters**: Always consider the economics of your AI choices
- 🔧 **Control is valuable**: Being able to customize behavior is crucial in production

## Summary of Changes

| Aspect | 2024 Approach | 2025 Modern Approach | Why Changed |
|--------|---------------|---------------------|-------------|
| **Deployment** | LangServe | Direct FastAPI | LangServe deprecated |
| **Embeddings** | text-embedding-ada-002 | text-embedding-3-small | 5x cheaper, better performance |
| **Python** | 3.11.4 | 3.13.3 | Performance, security, features |
| **LLM** | gpt-4-1106-preview | gpt-4o-mini | Much cheaper, RAG-optimized |
| **Chain Structure** | Complex with TypedDict | Simple with RunnablePassthrough | Cleaner, more readable |
| **Dependencies** | Many constraints | Clean, modern versions | Fewer conflicts |
| **Error Handling** | Limited (black box) | Full control | Better debugging |
| **Document Processing** | Required flattening | Direct processing | Bug fixes in libraries |

## Next Steps

Now that you understand what changed and why, let's move to:
- **nbv2-part1c-implementation.ipynb** - Step-by-step implementation of the modern version
- **nbv2-part1d-comparison.ipynb** - Side-by-side code comparison